## TYTAN tutorial おすすめ3（数字を均等に2組に分ける）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で複数の数字を均等に2組に分ける」を解く](https://vigne-cla.com/21-25/)

### 問題

次の6つの自然数を、総和が等しくなるように2グループに分けなさい。<br>
15, 25, 33, 41, 64, 82


<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-25_bmeiqbd.jpg" width = 45%>
</div>

入試問題として出題される場合は倍数、余りなど整数の性質を利用して解けるようになっている。さすがに入試で全探索（力まかせ探索）は出ないと信じる・・・

### このQUBO設定を使おう

<font color="red">「解が0と1だけの方程式」</font>

「重さ1, 2, 3のボールのどれを取れば合計3の重さになるか？」のは次のように設定できる。
```
H = (1*q0 + 2*q1 + 3*q2 - 3)**2
```

解は [q0, q1, q2] = [0, 0, 1] or [1, 1, 0] となり、1, 2のボール、または3のボールを取れば良いという結果になる。

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

まず、すべての数字の合計が260なので各グループは合計130である。

数字に6個の量子ビット（q0～q5）を割り当て、1になったらグループA、0になったらグループBとする。

グループAの量子ビットは1であることを利用し、重みとして数字をかけた合計が130になるような式を設定する。<font color="red">「量子ビットが1になったときにその数字の重みが有効」</font>になる。

```
#グループA（＝１になった量子ビット）は合計130
H = (15*q0 + 25*q1 + 33*q2 + 41*q3 + 64*q4 + 82*q5 - 130)**2
```

反対に、グループBの量子ビットは0であることから、こちらも重みをかけたものの合計が130になるように設定する。ここでテクニックとして、**q0 を (1 – q0) と置き換える**ことで０，１の条件をひっくり返し<font color="red">「0になったときに重みが有効」</font>を表現できる。

```
#グループB（＝０になった量子ビット）は合計130
H = (15*(1-q0) + 25*(1-q1) + 33*(1-q2) + 41*(1-q3) + 64*(1-q4) + 82*(1-q5) - 130)**2
```

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [1]:
from tytan import symbols, symbols_list, Compile, sampler

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')

#グループA（＝１になった量子ビット）は合計130
H = 0
H += (15*q0 + 25*q1 + 33*q2 + 41*q3 + 64*q4 + 82*q5 - 130)**2

#グループB（＝０になった量子ビット）は合計130
H += (15*(1-q0) + 25*(1-q1) + 33*(1-q2) + 41*(1-q3) + 64*(1-q4) + 82*(1-q5) - 130)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#結果
for r in result:
    print(r)

offset
33800
[{'q0': 0, 'q1': 1, 'q2': 0, 'q3': 1, 'q4': 1, 'q5': 0}, -33800.0, 28]
[{'q0': 1, 'q1': 0, 'q2': 1, 'q3': 0, 'q4': 0, 'q5': 1}, -33800.0, 28]
[{'q0': 0, 'q1': 0, 'q2': 0, 'q3': 1, 'q4': 0, 'q5': 1}, -33702.0, 21]
[{'q0': 1, 'q1': 1, 'q2': 1, 'q3': 0, 'q4': 1, 'q5': 0}, -33702.0, 23]


最適解は2通り出るがグループ名を入れ替えたものなので同じ。

[15, 25, 33, 41, 64, 82] => [0, 1, 0, 1, 1, 0] ということで、グループAに [25, 41, 64]、グループBに [15, 33, 82] と分かった。

気がついたかもしれないが実はグループBの設定式は無くてもOK。グループAの条件を満たしていれば必然的にグループBの条件も満たすから。

グループBの条件式をコメントアウトして実行してみよう。

In [2]:
import numpy as np

numbers = np.array([15, 25, 33, 41, 64, 82])

q = symbols_list(len(numbers))
z = 2*q - 1  # イジング変数

# -1 と +1 の個数が同じ
# -1 のイジング変数も +1 のイジング変数も、担当する数字の和がバランスする
H = sum(z)**2 + sum(numbers * z)**2

qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

solver = sampler.SASampler()

result = solver.run(qubo)

for r in result:
    print(r)

offset
67636
[{'q0': 0, 'q1': 1, 'q2': 0, 'q3': 1, 'q4': 1, 'q5': 0}, -67636.0, 24]
[{'q0': 1, 'q1': 0, 'q2': 1, 'q3': 0, 'q4': 0, 'q5': 1}, -67636.0, 23]
[{'q0': 0, 'q1': 0, 'q2': 0, 'q3': 1, 'q4': 0, 'q5': 1}, -67436.0, 25]
[{'q0': 1, 'q1': 1, 'q2': 1, 'q3': 0, 'q4': 1, 'q5': 0}, -67436.0, 28]
